# マルチメディア処理入門 第11回
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/schwalbe1996/multimedia_intro/blob/main/notebook11.ipynb)

Google Colabで試したい場合は、上のボタンをクリックして、「ドライブにコピー」を実行してください。（ドライブにコピーしないとコードを変更しても保存できません）

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pylab import cm
import cv2

## 画像ファイルの読み込み
- このノートブックと同じフォルダに画像ファイル(tomato_sample.png)を置いてください。
- 今日はモノクロ画像として扱います

In [ ]:
# colabユーザだけここを実行
!wget https://github.com/schwalbe1996/multimedia_intro/raw/main/tomato_sample.png

In [ ]:
image = cv2.imread("tomato_sample.png",0)

## 画像表示用の関数

In [ ]:
# opencvではカラー画像の表現がBGR（青・緑・赤）の順番なのに対して、matplotlibではRGB（赤・緑・青）の順番なので、
# plt.imshow(image)ではなく、plt.imshow(image[:,:,::-1])とやってBGR⇒RGBに順番を入れ替えておく
def show_image(data,title=None):
    plt.figure(figsize=(10,10))
    if data.ndim == 3:
        plt.imshow(data[:,:,::-1])
    else:
        plt.imshow(data,cmap=cm.gray)
        plt.colorbar()
    plt.title(title)
    plt.show()

In [ ]:
# 講義資料のように3次元プロットしたい場合はこちらも実行してください
from mpl_toolkits.mplot3d import Axes3D

def show_image3d(data,title=None):
    x = np.arange(data.shape[1])
    y = np.arange(data.shape[0])
    xx,yy = np.meshgrid(x,y)

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=30, azim=30) # 3次元の視点角を設定します。

    #ax.set_zlim(-2,8) # ｚ軸の範囲を設定したい場合はコメントを外してください   

    sp = ax.plot_surface(xx,yy,data,cmap=cm.jet,vmin=None,vmax=None)
    fig.colorbar(sp,shrink=0.5)

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    #ax.set_zlabel("log(power)")

    fig.show()    

## 下準備　: 画像にノイズを加える
- 平均0，分散適当のガウス分布に従うノイズを用意します。

In [ ]:
show_image(image, "original")

noise = np.random.normal(0, 15, image.shape)
show_image(noise, "noise")

src = noise + image
# 0〜255の範囲に収まってない画素は 0 or 255にしておく
src = np.clip(src, 0, 255)
src = src.astype(np.uint8)
show_image(src, "source image")


## 離散フーリエ変換してみましょう

### まずは元の画像に対して離散フーリエ変換

In [ ]:
freq = np.fft.fft2(image)

# 離散フーリエ変換した結果は複素数になるので、可視化のために対数パワースペクトルに変換
power_spectrum = np.log( np.abs(freq) )
plt.imshow(power_spectrum)

In [ ]:
# 上の結果だと低周波成分が図中の端にきて見にくいので、低周波成分が図の中央に寄るように加工します
power_spectrum = np.fft.fftshift(power_spectrum)
plt.imshow(power_spectrum)


In [ ]:
# 3D表示したい人だけ
show_image3d(power_spectrum)

### 離散逆フーリエ変換

In [ ]:
rev_image = np.fft.ifft2(freq)

# 逆フーリエ変換した結果も複素数になるので、絶対値を取ってます。
rev_image = np.abs(rev_image)
show_image(rev_image)

### 画像⇒平均値フィルタ
- 前回行った、3x3の平均値フィルタを実行した結果
- 今回の講義で説明した方法
    1. (ノイズ入り）画像を離散フーリエ変換
    2. 3x3平均値フィルタも離散フーリエ変換
    3. (1)×(2)を計算
    4. (3)の結果を逆離散フーリエ変換

In [ ]:
# まずは3x3の平均値フィルタを畳み込む処理
output = cv2.blur(src, ksize=(3,3))
show_image(output, "mean filter (3x3)")

In [ ]:
# ノイズ入り画像を離散フーリエ変換して、パワースペクトルを表示してみましょう
freq_noisy = np.fft.fft2(src)
power_spectrum = np.log( np.abs(freq_noisy) )
power_spectrum = np.fft.fftshift(power_spectrum)
plt.imshow(power_spectrum)
plt.colorbar()

In [ ]:
# 3x3の平均値フィルタを離散フーリエ変換して、パワースペクトルを表示してみましょう
mfilter = np.zeros(src.shape)
mfilter[-1,-1] = 1/9
mfilter[-1,0] = 1/9
mfilter[-1,1] = 1/9
mfilter[0,-1] = 1/9
mfilter[0,0] = 1/9
mfilter[0,1] = 1/9
mfilter[1,-1] = 1/9
mfilter[1,0] = 1/9
mfilter[1,1] = 1/9

freq_mean = np.fft.fft2(mfilter)
power_spectrum = np.abs(freq_mean)
power_spectrum = np.fft.fftshift(power_spectrum)
plt.imshow(power_spectrum)
plt.colorbar()

In [ ]:
# 周波数領域で画像とフィルタを掛け算
freq_dst = freq_noisy * freq_mean
power_spectrum = np.log( np.abs(freq_dst) + 1e-10 ) # log(0)がエラーになるのですごく小さい数を足してます。
power_spectrum[ power_spectrum == np.log(1e-10) ] = np.nan
power_spectrum = np.fft.fftshift(power_spectrum)
plt.imshow(power_spectrum)
plt.colorbar()

In [ ]:
# 周波数領域で画像とフィルタを掛け算した結果を逆離散フーリエ変換
dst = np.fft.ifft2(freq_dst)

# 逆フーリエ変換した結果も複素数になるので、絶対値を取ってます。
dst = np.abs(dst)
show_image(dst)